<a href="https://colab.research.google.com/github/Marcelooyola/Modulo6/blob/main/trabajo_final_mod6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo](https://github.com/cristiandarioortegayubro/BA/blob/main/dba.png?raw=true)

# **Trabajo Final - Módulo 6 - Aprendizaje Automático - Machine Learning - Clasificación**

![logo](https://www.python.org/static/community_logos/python-powered-w-100x40.png)

~~~
Apellido: Oyola
Nombre: Marcelo
~~~

**En virtud del DataFrame propuesto, generar modelos de clasificación para predecir la contratación de depositos a plazo:**

- **age:** edad del cliente, valor numérico entero mayor a 18.
- **job:** trabajo del cliente.
- **marital:** estado civil del cliente.
- **education:** educación alcanzada por el cliente.
- **default:** si el cliente posee deudas.
- **balance:** depósito del cliente en euros.
- **housing:** propietario de inmueble o alquila.
- **loan:** si el cliente tiene un préstamo.
- **contact:** último contacto del cliente.
- **day:** día del último contacto del cliente.
- **month:** mes del último contacto el cliente.
- **duration:** duración del último contacto del cliente.
- **campaign:** identificación de la campaña.
- **pdays:** dias posteriores a la campaña.
- **previous:** números de contactos previos a la campaña.
- **poutcome:** resultados de la campaña anterior.
- **y:** si suscribe un depósito a plazo posterior a campaña.

# **Carga de módulos y datos necesarios**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder # codificar variables categoricas
from sklearn.preprocessing import StandardScaler # estandarizar variables numericas
from sklearn.model_selection import train_test_split

In [30]:
#regresion logistica
from sklearn.linear_model import LogisticRegression
#arbol de regresion
from sklearn.ensemble import RandomForestClassifier


# **Creación del DataFrame**

In [3]:
url="https://raw.githubusercontent.com/cristiandarioortegayubro/BA/main/Datasets/"
archivo="bank-full.csv"
clasificacion = pd.read_csv(url+archivo)

In [4]:
clasificacion.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN,no


In [5]:
clasificacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        44923 non-null  object
 2   marital    45211 non-null  object
 3   education  43354 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    32191 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   8252 non-null   object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
clasificacion.isna().sum().sort_values(ascending = False)

poutcome     36959
contact      13020
education     1857
job            288
month            0
previous         0
pdays            0
campaign         0
duration         0
age              0
day              0
loan             0
housing          0
balance          0
default          0
marital          0
y                0
dtype: int64

In [7]:
#Funcion para ver mejor el contenido de cada columna

for i in clasificacion.columns:
  print("------"*10)
  print("Datos de Columna: "+ i)
  print(clasificacion[i].unique())
  print("Cantidad de datos nulos: "+ str(clasificacion[i].isna().sum()))
  print("Cantidad total de datos: "+ str((clasificacion[i].count() + clasificacion[i].isna().sum())))

------------------------------------------------------------
Datos de Columna: age
[58 44 33 47 35 28 42 43 41 29 53 57 51 45 60 56 32 25 40 39 52 46 36 49
 59 37 50 54 55 48 24 38 31 30 27 34 23 26 61 22 21 20 66 62 83 75 67 70
 65 68 64 69 72 71 19 76 85 63 90 82 73 74 78 80 94 79 77 86 95 81 18 89
 84 87 92 93 88]
Cantidad de datos nulos: 0
Cantidad total de datos: 45211
------------------------------------------------------------
Datos de Columna: job
['management' 'technician' 'entrepreneur' 'blue-collar' nan 'retired'
 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid' 'student']
Cantidad de datos nulos: 288
Cantidad total de datos: 45211
------------------------------------------------------------
Datos de Columna: marital
['married' 'single' 'divorced']
Cantidad de datos nulos: 0
Cantidad total de datos: 45211
------------------------------------------------------------
Datos de Columna: education
['tertiary' 'secondary' nan 'primary']
Cantidad de datos nulos: 1857
Ca

##Variables categoricas y numericas

In [8]:
#numericas
cols_numericas = list(clasificacion.select_dtypes(exclude=["object"]))
cols_numericas

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [9]:
#categoricas
cols_categoricas = list(clasificacion.select_dtypes(include=["object"]))
cols_categoricas

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'y']

##Imputaciones

poutcome     36959
contact      13020
education     1857
job            288

In [10]:
clasificacion.poutcome

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
45206        NaN
45207        NaN
45208    success
45209        NaN
45210      other
Name: poutcome, Length: 45211, dtype: object

In [11]:
imputacion = SimpleImputer(strategy = 'most_frequent')
cols_imputar = ['poutcome', 'contact', 'education','job']

for i in cols_imputar:
  clasificacion[i] = imputacion.fit_transform(clasificacion[['poutcome']])

#clasificacion.poutcome = imputacion.fit_transform(clasificacion[['poutcome']])
#clasificacion.poutcome

In [12]:
clasificacion

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,failure,married,failure,no,2143,yes,no,failure,5,may,261,1,-1,0,failure,no
1,44,failure,single,failure,no,29,yes,no,failure,5,may,151,1,-1,0,failure,no
2,33,failure,married,failure,no,2,yes,yes,failure,5,may,76,1,-1,0,failure,no
3,47,failure,married,failure,no,1506,yes,no,failure,5,may,92,1,-1,0,failure,no
4,33,failure,single,failure,no,1,no,no,failure,5,may,198,1,-1,0,failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,failure,married,failure,no,825,no,no,failure,17,nov,977,3,-1,0,failure,yes
45207,71,failure,divorced,failure,no,1729,no,no,failure,17,nov,456,2,-1,0,failure,yes
45208,72,success,married,success,no,5715,no,no,success,17,nov,1127,5,184,3,success,yes
45209,57,failure,married,failure,no,668,no,no,failure,17,nov,508,4,-1,0,failure,no


In [13]:
clasificacion.isnull().sum().sort_values(ascending = False)

age          0
day          0
poutcome     0
previous     0
pdays        0
campaign     0
duration     0
month        0
contact      0
job          0
loan         0
housing      0
balance      0
default      0
education    0
marital      0
y            0
dtype: int64

##Codificar variables categoricas

In [14]:
clasificacion.select_dtypes(include=["object"])

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,failure,married,failure,no,yes,no,failure,may,failure,no
1,failure,single,failure,no,yes,no,failure,may,failure,no
2,failure,married,failure,no,yes,yes,failure,may,failure,no
3,failure,married,failure,no,yes,no,failure,may,failure,no
4,failure,single,failure,no,no,no,failure,may,failure,no
...,...,...,...,...,...,...,...,...,...,...
45206,failure,married,failure,no,no,no,failure,nov,failure,yes
45207,failure,divorced,failure,no,no,no,failure,nov,failure,yes
45208,success,married,success,no,no,no,success,nov,success,yes
45209,failure,married,failure,no,no,no,failure,nov,failure,no


In [15]:
clasificacion[cols_categoricas] = clasificacion[cols_categoricas].apply(LabelEncoder().fit_transform)

In [16]:
clasificacion.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,0,1,0,0,2143,1,0,0,5,8,261,1,-1,0,0,0
1,44,0,2,0,0,29,1,0,0,5,8,151,1,-1,0,0,0
2,33,0,1,0,0,2,1,1,0,5,8,76,1,-1,0,0,0
3,47,0,1,0,0,1506,1,0,0,5,8,92,1,-1,0,0,0
4,33,0,2,0,0,1,0,0,0,5,8,198,1,-1,0,0,0


## Estandarizar variables numericas

In [17]:
clasificacion[cols_numericas].head(5)

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0


In [18]:
scaler = StandardScaler()

In [19]:
clasificacion[cols_numericas] = scaler.fit_transform(clasificacion[cols_numericas])

In [20]:
clasificacion[cols_numericas].sample(5)

,age,balance,day,duration,campaign,pdays,previous
26204,1.136095,-0.445120,0.503892,-0.738426,-0.569351,-0.411453,-0.251940
7467,-1.029906,-0.406365,1.585313,-0.078296,-0.246560,-0.411453,-0.251940
13721,-0.559037,-0.010599,-0.697687,2.496212,-0.569351,-0.411453,-0.251940
23338,0.288529,0.387137,1.344997,-0.466608,0.076230,-0.411453,-0.251940
28087,1.230269,-0.345604,1.465155,1.040043,0.399020,2.175245,1.050473


## Preparar el dataset

In [21]:
X = clasificacion.drop(labels = ['y'],axis = 1)
X

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,1.606965,0,1,0,0,0.256419,1,0,0,-1.298476,8,0.011016,-0.569351,-0.411453,-0.251940,0
1,0.288529,0,2,0,0,-0.437895,1,0,0,-1.298476,8,-0.416127,-0.569351,-0.411453,-0.251940,0
2,-0.747384,0,1,0,0,-0.446762,1,1,0,-1.298476,8,-0.707361,-0.569351,-0.411453,-0.251940,0
3,0.571051,0,1,0,0,0.047205,1,0,0,-1.298476,8,-0.645231,-0.569351,-0.411453,-0.251940,0
4,-0.747384,0,2,0,0,-0.447091,0,0,0,-1.298476,8,-0.233620,-0.569351,-0.411453,-0.251940,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.947747,0,1,0,0,-0.176460,0,0,0,0.143418,9,2.791329,0.076230,-0.411453,-0.251940,0
45207,2.831227,0,0,0,0,0.120447,0,0,0,0.143418,9,0.768224,-0.246560,-0.411453,-0.251940,0
45208,2.925401,2,1,2,0,1.429593,0,0,2,0.143418,9,3.373797,0.721811,1.436189,1.050473,2
45209,1.512791,0,1,0,0,-0.228024,0,0,0,0.143418,9,0.970146,0.399020,-0.411453,-0.251940,0


In [22]:
y = clasificacion['y']
y

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64

In [23]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state = 123)

# **Generación del primer modelo de clasificación**

In [25]:
#crear y entrenar modelo
modelo_regresion = LogisticRegression()
modelo_regresion.fit(X_train, y_train)

LogisticRegression()

In [26]:
#hacer prediccion sobre conjunto de pruebas
y_prediccion = modelo_regresion.predict(X_test)

In [27]:
#Evaluar modelo
accuracy = accuracy_score(y_test, y_prediccion)
reporte_clasificacion = classification_report(y_test, y_prediccion)

print(f"Precision: {accuracy}")
print(f"Reporte de clasificacion: \n {reporte_clasificacion}")

Precision: 0.8980389265703332
Reporte de clasificacion: 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94     11998
           1       0.62      0.30      0.41      1566

    accuracy                           0.90     13564
   macro avg       0.77      0.64      0.68     13564
weighted avg       0.88      0.90      0.88     13564



# **Generación del segundo modelo de clasificación**

In [31]:
modelo_arbol = RandomForestClassifier()
modelo_arbol.fit(X_train, y_train)

RandomForestClassifier()

In [32]:
y_prediccion = modelo_arbol.predict(X_test)

In [33]:
from sklearn.metrics import classification_report
accuracy = accuracy_score(y_test, y_prediccion)
reporte_clasificacion  = classification_report(y_test, y_prediccion)

print(f"Precision: {accuracy}")
print(f"Reporte de clasificacion: \n {reporte_clasificacion}")

Precision: 0.9031996461220879
Reporte de clasificacion: 
               precision    recall  f1-score   support

           0       0.93      0.97      0.95     11998
           1       0.62      0.41      0.49      1566

    accuracy                           0.90     13564
   macro avg       0.77      0.69      0.72     13564
weighted avg       0.89      0.90      0.89     13564



# **Conclusiones**

El reporte de clasificación proporciona una evaluación detallada del rendimiento del modelo de clasificación. Vamos a analizar los diferentes aspectos del reporte:

1. **Precision**:
   - Para la clase 0: 0.93. Esto significa que de todas las predicciones que el modelo etiquetó como clase 0, el 93% de ellas eran realmente de la clase 0.
   - Para la clase 1: 0.62. Esto significa que de todas las predicciones que el modelo etiquetó como clase 1, el 62% de ellas eran realmente de la clase 1.

2. **Recall**:
   - Para la clase 0: 0.97. Esto significa que el 97% de las muestras reales de la clase 0 fueron identificadas correctamente por el modelo.
   - Para la clase 1: 0.41. Esto significa que solo el 41% de las muestras reales de la clase 1 fueron identificadas correctamente por el modelo.

3. **F1-Score**:
   - Para la clase 0: 0.95. El F1-score es la media armónica de la precisión y el recall, lo que proporciona una medida equilibrada entre ambas métricas. Un valor alto indica un buen equilibrio entre precision y recall.
   - Para la clase 1: 0.49. El F1-score de la clase 1 es relativamente bajo, lo que indica que el modelo tiene dificultades para clasificar correctamente las muestras de esta clase.

4. **Support**:
   - Para la clase 0: 11998. Este es el número total de muestras en la clase 0 en tu conjunto de prueba.
   - Para la clase 1: 1566. Este es el número total de muestras en la clase 1 en tu conjunto de prueba.

5. **Accuracy**:
   - 0.90. La precisión global del modelo en el conjunto de prueba.

6. **Macro avg**:
   - Precisión promedio para todas las clases (0.77).
   - Recall promedio para todas las clases (0.69).
   - F1-Score promedio para todas las clases (0.72).

7. **Weighted avg**:
   - Estas son las mismas métricas promediadas, pero ponderadas por el número de muestras en cada clase. Esto es útil cuando hay desbalance de clases.

En general, el modelo tiene una buena precisión en la clase 0, pero tiene dificultades para clasificar correctamente la clase 1. Esto puede indicar que el modelo necesita ajustes adicionales, como el ajuste de hiperparámetros o incluso la consideración de técnicas de manejo de desbalance de clases si este es el caso.